In [3]:
import numpy as np
import pandas as pd
RI = np.array([0,0,0.58,0.90,1.12,1.24,1.32,1.41,1.45,1.49,1.51,1.48,1.56,1.57,1.58])

#subcriteria
df =pd.read_csv("WesternGhatData/1Elevation.csv")
elevation= df.to_numpy()

df =pd.read_csv("WesternGhatData/2Slope.csv")
slope= df.to_numpy()

df =pd.read_csv("WesternGhatData/3DistancefromRiver.csv")
river= df.to_numpy()

df =pd.read_csv("WesternGhatData/4Drainage.csv")
drainage= df.to_numpy()

df =pd.read_csv("WesternGhatData/5FlowAccumulation.csv")
flow= df.to_numpy()

df =pd.read_csv("WesternGhatData/6TWI.csv")
twi= df.to_numpy()

df =pd.read_csv("WesternGhatData/7Rainfall.csv")
rainfall= df.to_numpy()

df =pd.read_csv("WesternGhatData/8Landuse.csv")
landuse= df.to_numpy()

df =pd.read_csv("WesternGhatData/9SoilTexture.csv")
soil= df.to_numpy()

df =pd.read_csv("WesternGhatData/10TRI.csv")
tri= df.to_numpy()

df =pd.read_csv("WesternGhatData/11Geology.csv")
geology= df.to_numpy()

data = np.array([[1   ,2   ,2   ,3   ,4   ,5   ,5   ,6   ,7   ,8   ,8],
                   [1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,6   ,7   ,8   ,8],
                   [1/2 ,1/2 ,1   ,2   ,3   ,4   ,4   ,5   ,6   ,7   ,7],
                   [1/3 ,1/3 ,1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,7   ,8],
                   [1/4 ,1/3 ,1/3 ,1/2 ,1   ,2   ,2   ,3   ,4   ,5   ,6],
                   [1/5 ,1/4 ,1/4 ,1/3 ,1/2 ,1   ,2   ,4   ,5   ,7   ,7],
                   [1/5 ,1/5 ,1/4 ,1/3 ,1/2 ,1/2 ,1   ,2   ,3   ,5   ,6],
                   [1/6 ,1/6 ,1/5 ,1/4 ,1/3 ,1/4 ,1/2 ,1   ,2   ,4   ,5],
                   [1/7 ,1/7 ,1/6 ,1/5 ,1/4 ,1/5 ,1/3 ,1/2 ,1   ,3   ,4],
                   [1/8 ,1/8 ,1/7 ,1/7 ,1/5 ,1/7 ,1/5 ,1/4 ,1/3 ,1   ,3],
                   [1/8 ,1/8 ,1/7 ,1/8 ,1/6 ,1/7 ,1/6 ,1/5 ,1/4 ,1/3 ,1]]
                  )

n = data[0].size
float_formatter = "{:.3f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

In [4]:
#converts normal into fuzzyarray
def fuzzify(data):
    n = data[0].size
    mainarr = np.full((n,n,3),0,dtype=float)
    for i in range(0,n):
        for j in range(0,n):
            if (data[i][j]>=1):
                mainarr[i][j][0]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]-1
                mainarr[i][j][2]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]+1
            else:
                for k in range(2,10):
                    if (1/k==data[i][j]):
                        mainarr[i][j][0]= data[i][j] if(data[i][j]==9) else 1/(k+1)
                        mainarr[i][j][2]= data[i][j] if(data[i][j]==9) else 1/(k-1)
            mainarr[i][j][1]= data[i][j]
    return mainarr

In [5]:
def criteriaWeights(mainarr):
    n = mainarr[0].size
    colsum = mainarr.sum(axis=0)
    normalizedform = np.array(mainarr)
    for i in range(0,n):
        normalizedform[i] = np.divide(mainarr[i] , colsum)
    criteriaweights = normalizedform.sum(axis=1)/n
    return criteriaweights

def FuzzycriteriaWeights(mainarr):
    n = mainarr[0].size if (np.ndim(mainarr)==2) else int(mainarr[0].size/3)
    fuzzyGM = np.full((n,3),1,dtype=float)
    for i in range(0,n):
        for j in range(0,n):
            fuzzyGM[i] = np.multiply(fuzzyGM[i],mainarr[i][j])
        fuzzyGM[i]=np.power(fuzzyGM[i],1/n)
    fuzzyWeights=np.full((n,3),1,dtype=float)
    for i in range(0,n):
        fuzzyWeights[i]=np.multiply(fuzzyGM[i],np.reciprocal(np.flip(fuzzyGM.sum(axis=0),0)))
    weights = np.average(fuzzyWeights,axis=1)
    weights = np.divide(weights,weights.sum(axis=0))
    return weights

In [71]:
def consistency(mainarr):
    n = int(mainarr[0].size)
    fuzzyArr = fuzzify(mainarr)
    c1 = np.array(mainarr)
    fc1 = np.array(mainarr)
    for i in range(0,n):
        c1[i] = np.multiply(mainarr[i],criteriaWeights((mainarr)))
        fc1[i] = np.multiply(mainarr[i],FuzzycriteriaWeights((mainarr)))
    lamda = np.divide(c1.sum(axis=1),criteriaWeights((mainarr))).sum(axis=0)/n
    flamda = (np.divide((fc1.sum(axis=1)),FuzzycriteriaWeights((mainarr)))).sum(axis=0)/n
    ci = (lamda-n)/(n-1)
    fci = (flamda-n)/(n-1)
    CR = ci / RI[n-1]
    FCR = fci / RI[n-1]

    if(CR<0.1 and FCR<0.1):
        print("CR: ",CR," VALUES ARE CONSISTENT - go ahead")
        print("cwAHP",criteriaWeights(mainarr))
        print("FUZZY CR: ",FCR," VALUES ARE CONSISTENT - go ahead")
        print("cwfuzzyAHP",FuzzycriteriaWeights(fuzzyArr))
    else:
        print("CR: ",CR,"criteria weights are not consistent reEnter the values")
        print("FUZZY CR: ",FCR," VALUES ARE CONSISTENT - go ahead")

In [ ]:
consistency(data)

In [ ]:
print(consistency(elevation),"elevation\n\n")
print(consistency(slope),"slope\n\n")
print(consistency(river),"river\n\n")
print(consistency(drainage),"drainage\n\n")
print(consistency(flow),"flow\n\n")
print(consistency(twi),"twi\n\n")
print(consistency(rainfall),"rainfall\n\n")
print(consistency(landuse),"landuse\n\n")
print(consistency(soil),"soil\n\n")
print(consistency(tri),"tri\n\n")
print(consistency(geology),"geology\n\n")